In [1]:
from sub_func import *
from pf_selection import *
from pipeline_utils import *

from crawl_tradingview import *
from t1 import *

from datetime import datetime, timedelta

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# 1,013,680 -> 3,559,857

# 오늘 날짜 (코스피 장이 열린 날만 한정, 예시)
today = '20230102'

# 조회할 정보의 기간
base_year = '2022'
base_quarter = 'Q4'
start_date = '20221001'
end_date = '20221231'

# 포트폴리오를 활용하는 실제 기간 (ex. 2022 Q4를 조회하여 2023 Q1을 위한 포트폴리오 생성)
target_year = '2023'
target_quarter = 'Q1'

# 'pf_selection_agent'에서 종목 코드 가져오기
tickers = get_tickers_from_json('pf_selection_agent', f'{target_year}_{target_quarter}_init_pf')

json_file_path = './notion_page_ids.json'
data = read_json(json_file_path)
t_1_trader_report_id = data['t_1'][f'{today}_t_1_trader_report']
t_1_trader_report = get_all_text_from_page(t_1_trader_report_id)

trader_output_format = """{
    "log": "{매수/매도} | {수량} | {가격}"
}"""

trader_prompt = f"""당신은 주식 트레이더입니다.
주어진 데이터를 분석하여 매수 또는 매도, 관망 여부를 결정하고 거래 로그를 작성해야 합니다.
이때, 이전에 저장된 t1의 트레이더 output을 참고하여 더욱 정확한 거래를 해야 합니다.

만약 매수 또는 매도를 진행한다면, 다음 양식을 따라야 합니다: {trader_output_format}
만약 관망을 진행한다면, "관망"이라고 작성해야 합니다.

추가적인 단어 생성 없이, 반드시 {trader_output_format}에 따라 dict만을 작성하거나, "관망"이라고 작성해야 합니다.

당신의 역할은 수익성을 극대화하기 위해 정확하고 신속한 거래 결정을 내리는 것입니다.
거래 로그는 반드시 지정된 형식을 따라야 합니다.

t1의 트레이더 output: {t_1_trader_report}
"""

pf_path = f'./pf_logs/{target_year}_{target_quarter}/{today}_portfolio_weights.json'
pf_before_update = read_json(pf_path)

pf_update_system = """주어진 다음 거래 로그를 바탕으로 포트폴리오를 정확히 업데이트 하세요.
추가적인 글이나 json delimiter 따위를 생성하지 말고, output을 바로 json으로 저장할 수 있도록 출력하세요.

포트폴리오는 반드시 {ticker: weight} 형식을 따라야 합니다. 이때, 모든 종목의 weight의 합은 반드시 1이어야 합니다.
"""

In [ ]:
sp_dict = {}

# t2_trader
for i, ticker in enumerate(tickers):
    print(f'=== {i+1}/{len(tickers)} ===')

    try:
        sp_dict[ticker] = {}

        sp = stock_price_info(ticker, start_date, today)[['Close', 'RSI_14', 'BBP_20_2.0']]
        date_obj = datetime.strptime(today, "%Y%m%d")
        new_date_obj = date_obj + timedelta(days=10)
        end_day = new_date_obj.strftime("%Y%m%d")
        sp_dict[ticker]['sp'] = sp

        df = scrape_tradingview_data(ticker, today, end_day)
        filtered_df = df[df['Timestamp'].dt.strftime('%Y%m%d') == today]
        sp_dict[ticker]['hourly_sp'] = filtered_df
    except Exception as e:
        print(f'{e} | Error occurred at {ticker}')
        continue    

curr_hour = 9
t_2_trader_response = {}

while curr_hour < 16:
    t_2_trader_response[curr_hour] = {}

    for i, ticker in enumerate(tickers):
        print(f'=== [{curr_hour}] {i+1}/{len(tickers)} ({ticker}) ===')

        sp = sp_dict[ticker]['sp'] if ticker in sp_dict and 'sp' in sp_dict[ticker] else '역대 가격 정보를 가져올 수 없습니다.'

        # `sp_dict[ticker]['hourly_sp']`이 존재하면 DataFrame을 필터링하여 바인딩
        if ticker in sp_dict and 'hourly_sp' in sp_dict[ticker]:
            df = sp_dict[ticker]['hourly_sp']

            # curr_hour 이전의 데이터만 필터링
            filtered_df = df[df['Timestamp'].dt.hour < curr_hour]

            # 데이터가 존재하면 최신 데이터 가져오기 (Timestamp 기준 가장 최근 값)
            if not filtered_df.empty:
                hourly_sp = filtered_df  # 가장 최근 데이터 선택
            else:
                hourly_sp = '해당 시간 이전의 데이터가 없습니다.'
        else:
            hourly_sp = '실시간 정보를 가져올 수 없습니다.'

        price_info = f"""역대 주식 가격: {sp}
        실시간 주식 가격: {hourly_sp}"""

        response = to_GPT(trader_prompt, price_info)['choices'][0]['message']['content']
        t_2_trader_response[curr_hour][ticker] = response

    curr_hour += 1

to_DB('t_2', f'{today}_t_2_t_4_trader_log', f'{base_quarter}_{base_year}', str(t_2_trader_response))

pf_update_prompt = f"""
이전 포트폴리오: {pf_before_update}
"""

pf_update = to_GPT(pf_update_system, pf_update_prompt)['choices'][0]['message']['content']

# 파일 경로
file_path = os.path.join(f'./pf_logs/{target_year}_{target_quarter}', f"{today}_portfolio_weights.json")

# JSON 파일 저장
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(eval(pf_update), f, indent=2, ensure_ascii=False)

print(f"포트폴리오 비중 데이터가 {file_path}에 저장되었습니다.")
to_DB('t_2', f'{today}_t_2_t_4_pf_update', f'{base_quarter}_{base_year}', str(pf_update))

=== 1/22 ===
=== 2/22 ===
=== 3/22 ===
=== 4/22 ===
=== 5/22 ===
=== 6/22 ===
'Timestamp' | Error occurred at 300720
=== 7/22 ===
=== 8/22 ===
=== 9/22 ===
=== 10/22 ===
'Timestamp' | Error occurred at 000990
=== 11/22 ===
=== 12/22 ===
=== 13/22 ===
=== 14/22 ===
=== 15/22 ===
=== 16/22 ===
'Timestamp' | Error occurred at 000670
=== 17/22 ===
=== 18/22 ===
=== 19/22 ===
=== 20/22 ===
=== 21/22 ===
=== 22/22 ===


In [3]:
curr_hour = 9
t_2_trader_response = {}

while curr_hour < 16:
    t_2_trader_response[curr_hour] = {}

    for i, ticker in enumerate(tickers):
        print(f'=== [{curr_hour}] {i+1}/{len(tickers)} ({ticker}) ===')

        sp = sp_dict[ticker]['sp'] if ticker in sp_dict and 'sp' in sp_dict[ticker] else '역대 가격 정보를 가져올 수 없습니다.'

        # `sp_dict[ticker]['hourly_sp']`이 존재하면 DataFrame을 필터링하여 바인딩
        if ticker in sp_dict and 'hourly_sp' in sp_dict[ticker]:
            df = sp_dict[ticker]['hourly_sp']

            # curr_hour 이전의 데이터만 필터링
            filtered_df = df[df['Timestamp'].dt.hour < curr_hour]

            # 데이터가 존재하면 최신 데이터 가져오기 (Timestamp 기준 가장 최근 값)
            if not filtered_df.empty:
                hourly_sp = filtered_df  # 가장 최근 데이터 선택
            else:
                hourly_sp = '해당 시간 이전의 데이터가 없습니다.'
        else:
            hourly_sp = '실시간 정보를 가져올 수 없습니다.'

        price_info = f"""역대 주식 가격: {sp}
        실시간 주식 가격: {hourly_sp}"""

        response = to_GPT(trader_prompt, price_info)['choices'][0]['message']['content']
        t_2_trader_response[curr_hour][ticker] = response

    curr_hour += 1

to_DB('t_2', f'{today}_t_2_t_4_trader_log', f'{base_quarter}_{base_year}', str(t_2_trader_response))

pf_update_prompt = f"""
이전 포트폴리오: {pf_before_update}
"""

pf_update = to_GPT(pf_update_system, pf_update_prompt)['choices'][0]['message']['content']

# 파일 경로
file_path = os.path.join(f'./pf_logs/{target_year}_{target_quarter}', f"{today}_portfolio_weights.json")

# JSON 파일 저장
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(eval(pf_update), f, indent=2, ensure_ascii=False)

print(f"포트폴리오 비중 데이터가 {file_path}에 저장되었습니다.")
to_DB('t_2', f'{today}_t_2_t_4_pf_update', f'{base_quarter}_{base_year}', str(pf_update))

In [60]:
pf_update_prompt = f"""
이전 포트폴리오: {pf_before_update}
"""

pf_update = to_GPT(pf_update_system, pf_update_prompt)['choices'][0]['message']['content']

# 파일 경로
file_path = os.path.join(f'./pf_logs/{target_year}_{target_quarter}', f"{today}_portfolio_weights.json")

# JSON 파일 저장
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(eval(pf_update), f, indent=2, ensure_ascii=False)

print(f"포트폴리오 비중 데이터가 {file_path}에 저장되었습니다.")
to_DB('t_2', f'{today}_t_2_t_4_pf_update', f'{base_quarter}_{base_year}', str(pf_update))

In [ ]:
# 파일 경로
file_path = os.path.join(f'./pf_logs/{target_year}_{target_quarter}', f"{today}_portfolio_weights.json")

# JSON 파일 저장
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(eval(pf_update), f, indent=2, ensure_ascii=False)

print(f"포트폴리오 비중 데이터가 {file_path}에 저장되었습니다.")
to_DB('t_2', f'{today}_t_2_t_4_pf_update', f'{base_quarter}_{base_year}', str(pf_update))

포트폴리오 비중 데이터가 ./pf_logs/2023_Q1/20230102_portfolio_weights.json에 저장되었습니다.
페이지 생성 완료: 190cd049-9633-81b9-a6f6-ed55aaaa8f3a
텍스트 블럭 추가 완료
